### Querying Elasticsearch Using Python

In [ ]:
import elasticsearch
import unicodedata
import csv
from elasticsearch_dsl.aggs import Terms, Nested
from elasticsearch_dsl.search import Search
from elasticsearch_dsl import Q
import pandas as pd
from pandas.io.json import json_normalize
import json
import pickle
import datetime
import numpy as np

### Connect to Elasticsearch and Generate your Query

In [ ]:
es = elasticsearch.Elasticsearch(["elastic:elasticsiem@10.1.1.18:9200"])

#####################################################################
# Make your Query with elasticsearch_dsl
#####################################################################
res = Search(using=es, index="csv*").filter('range',**{"@timestamp":{'gte': '2019-08-08T00:00:00.000Z', 'lt' : '2019-08-09T23:59:59.999Z'}}).params(request_timeout=10000)#.sort("timestamp", {'order': "desc"})
# total = res.count()
# res = res[0:total]
#####################################################################
# Print the underlying JSON representation of your Query
#####################################################################
res_json = res.to_dict()
print(res_json)
    
#####################################################################
# Response execution
#####################################################################
response = res.execute()
#####################################################################

print("Filtered logs: %i" %res.count().value) #There is an error on the number of matched logs
print("Total number of logs: %d \n" %(response.hits.total.value))


### Test your data output to Pandas using json normalize

In [ ]:
#####################################################################
# Make a small test for normalizing a json response (single flow) from elasticsearch
#####################################################################
test_json = {'flow_id': 'EQQA////DP//////FP8BAAH6Fj4hHCz6Fj5kj2kKAQFBCgEBGLQTxuw', 'host': {'name': 'elk-master'}, 'start_time': '2018-10-23T23:47:34.605Z', '@timestamp': '2018-10-23T23:48:20.193Z', 'type': 'flow', 'source': {'ip': '10.1.1.24', 'port': 60614, 'stats': {'net_bytes_total': 14750, 'net_packets_total': 48}, 'mac': 'fa:16:3e:64:8f:69'}, 'final': False, 'tags': ['beats_input_raw_event'], 'dest': {'ip': '10.1.1.65', 'port': 5044, 'stats': {'net_bytes_total': 2424, 'net_packets_total': 24}, 'mac': 'fa:16:3e:21:1c:2c'}, 'beat': {'hostname': 'elk-master', 'name': 'elk-master', 'version': '6.4.2'}, 'transport': 'tcp', '@version': '1', 'last_time': '2018-10-23T23:48:17.797Z'}
pandas_test_df = json_normalize(test_json)
print(datetime.datetime.now())
# n_rows = len(pandas_df.index)
# if (n_rows % 1) == 0:
#     print(n_rows)
pandas_test_df.head()


### Save your data to a JSON file

In [ ]:
## # Normalize each response (flow in response) and append to dataframe
#####################################################################
print("#####################")
print(datetime.datetime.now())
print("#####################")
counter = 0
gonzalo_list =[]

for hits in res.scan():
    temp_json = hits.to_dict()
    temp_json['meta_id'] = hits.meta.id
    temp_json['meta_index'] = hits.meta.index
    #temp_json_pretty = json.dumps(temp_json, indent=4)
    gonzalo_list.append(temp_json)
    counter+=1
    if (counter % 1000) == 0:
        print(datetime.datetime.now())
        print(counter)
        #print("\n")
print(datetime.datetime.now())
print(counter)
with open('/home/ubuntu/my_test.json','w') as outfile:
    json.dump(gonzalo_list, outfile, ensure_ascii=False, indent=2)
    
#####################################################################
# Visualize the results
#####################################################################
#main_pandas_df.head(50)
outfile.close()

### Making detailed Queries

In [ ]:
es = elasticsearch.Elasticsearch(["elastic:elasticsiem@10.1.1.18:9200"])

#####################################################################
# Make your Query with elasticsearch_dsl
#####################################################################
res = Search(using=es, index="filebeat*").filter('range',**{"@timestamp":{'gte': '2018-10-12T00:00:00.000Z', 'lt' : '2019-03-29T23:59:59.999Z'}}).params(request_timeout=10000)\
.query("regexp", message="[0-9]{2,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}")\
.exclude("match", message='bound')\
.exclude('bool', filter=Q('match', message='ci')& Q('match', message='info'))\
.exclude("match", message='DataSourceOpenStack.py')\
.exclude("match", message='dhclient')\
.exclude("match", message='dhcp')\
.exclude("match", message='dhcp.py')\
.exclude("match", message='DHCPACK')\
.exclude("match", message='DHCPDISCOVER')\
.exclude("match", message='DHCPOFFER')\
.exclude("match", message='DHCPREQUEST')\
.exclude("match", message='ephemeral')\
.exclude("match", message='filebeat')\
.exclude("match", message='kibana')\
.exclude("match", message='launch_jupyter.sh')\
.exclude("match", message='ntpd')\
.exclude("match", message='packetbeat')\
.exclude("match", message='Resolving')\
.exclude("match", message='UFW')\
.exclude("match", message='url_helper.py')\
.query('bool', filter=Q('match', message='dhcpd')| Q('match', message='openstack.py')| Q('match', message='util.py'))

# .query("match", message='openstack.py')\
# .query("match", message='util.py')\
# .query("match", message='dhcpd')
#.query('bool', filter=Q('match', message='Received')& Q('match', message='dhcp lease'))
#.sort("@timestamp", {'order': "desc"})

# ES queries
# Tests
# message: /[0-9]{2,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}/ AND NOT message: "avahi-autoipd" AND NOT message: "bound" AND NOT message: "ci-info" AND NOT message: "cloud-init" AND NOT message: "DataSourceOpenStack.py" AND NOT message: "dhclient" AND NOT message: "dhcp" AND NOT message: "dhcp.py" AND NOT message: "DHCPACK" AND NOT message: "DHCPDISCOVER" AND NOT message: "DHCPOFFER" AND NOT message: "DHCPREQUEST" AND NOT message: "ephemeral" AND NOT message: "filebeat" AND NOT message: "kibana" AND NOT message: "launch_jupyter.sh" AND NOT message: "ntpd" AND NOT message: "packetbeat" AND NOT message: "Resolving URL: http:" AND NOT message: "UFW" AND NOT message: "url_helper.py"
# message: /[0-9]{2,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}/ AND NOT message: "bound" AND NOT message: "ci-info" AND NOT message: "cloud-init" AND NOT message: "DataSourceOpenStack.py" AND NOT message: "dhclient" AND NOT message: "dhcp" AND NOT message: "dhcp.py" AND NOT message: "DHCPACK" AND NOT message: "DHCPDISCOVER" AND NOT message: "DHCPOFFER" AND NOT message: "DHCPREQUEST" AND NOT message: "ephemeral" AND NOT message: "filebeat" AND NOT message: "kibana" AND NOT message: "launch_jupyter.sh" AND NOT message: "ntpd" AND NOT message: "packetbeat" AND NOT message: "Resolving URL: http:" AND NOT message: "UFW" AND NOT message: "url_helper.py"
# message: /[0-9]{2,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}/ AND NOT message: "bound" AND NOT message: "ci-info" AND NOT message: "DataSourceOpenStack.py" AND NOT message: "dhclient" AND NOT message: "dhcp" AND NOT message: "dhcp.py" AND NOT message: "DHCPACK" AND NOT message: "DHCPDISCOVER" AND NOT message: "DHCPOFFER" AND NOT message: "DHCPREQUEST" AND NOT message: "ephemeral" AND NOT message: "filebeat" AND NOT message: "kibana" AND NOT message: "launch_jupyter.sh" AND NOT message: "ntpd" AND NOT message: "packetbeat" AND NOT message: "Resolving URL: http:" AND NOT message: "UFW" AND NOT message: "url_helper.py"

# ES Query Final
# message: /[0-9]{2,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}/ AND NOT message: "bound" AND NOT message: "ci-info" AND NOT message: "DataSourceOpenStack.py" AND NOT message: "dhclient" AND NOT message: "dhcp" AND NOT message: "dhcpd" AND NOT message: "dhcp.py" AND NOT message: "DHCPACK" AND NOT message: "DHCPDISCOVER" AND NOT message: "DHCPOFFER" AND NOT message: "DHCPREQUEST" AND NOT message: "ephemeral" AND NOT message: "filebeat" AND NOT message: "kibana" AND NOT message: "launch_jupyter.sh" AND NOT message: "ntpd" AND NOT message:"openstack.py" AND NOT message: "packetbeat" AND NOT message: "Resolving URL: http:" AND NOT message: "UFW" AND NOT message: "url_helper.py" AND NOT message:"util.py"
# Test Matches
# .query("match", message="avahi-autoipd")
# .query("match", message="ntpd")
# .query("match", message="avahi-autoipd")
# .query("match", message="packetbeat")
# .query("match", message="filebeat")
# .query("match", message="packetbeat")
#.query("match", message="launch_jupyter")
#.query("match", message="dhclient")
#.query("match", message="dhcp.py")
#.query("match", message="ephemeral")
#.query("match", message="UFW")
#.query("match", message="url_helper.py")

# To be excluded
# AND NOT message:"url_helper.py"
# AND NOT message:"UFW"
# AND NOT message:"ephemeral"
# AND NOT message:"dhcp.py"
# AND NOT message:"dhclient"
# AND NOT message:"launch_jupyter.sh"
# AND NOT message:"packetbeat"
# AND NOT message:"filebeat"
# AND NOT message:"ntpd"
# AND NOT message:"dhcpd"
# AND NOT message:"cloud-init" # Removed since url_helper.py and ci-info where already removing cloud-init logs 
# and the exclude was eliminating messages with cloud and init words
# .exclude("match", message='cloud-init')\
# AND NOT message:"DHCPACK"
# AND NOT message:"kibana"
# AND NOT message:"ci-info" # converted to ci & info bool query since messages with the word info were being excluded
# AND NOT message:"DHCPDISCOVER"
# AND NOT message:"DHCPREQUEST"
# AND NOT message:"DHCPOFFER"
# AND NOT message:"DataSourceOpenStack.py"
# AND NOT message:"Resolving URL: http:""
# AND NOT message:"bound to" # bound to message is created by sd and dhclient programs

# Known Rules
# AND NOT message:"Failed"
# AND NOT message:"authentication"
# AND NOT message:"Disconnected"
# AND NOT message:"disconnect"
# AND NOT message:"closed"
# AND NOT message:"receive"
# AND NOT message:"reset"
# AND NOT message:"Invalid user"
# AND NOT message:"publickey"
# AND NOT message:"password"
# AND NOT message:"broken pipe"
# AND NOT message:"Unable to negotiate"
# AND NOT message:"COMMAND"
    
#Unknown
# AND NOT message:"Bad protocol version"
# AND NOT message:"Received dhcp lease" # Filtered out with dhcp.py
# AND NOT message:"Did not receive identification string"
# AND NOT message:"Protocol major versions differ for"
# AND NOT message:"No subnet declaration"
# AND NOT message:"Could not write ident string to"
# AND NOT message:"fatal: ssh_dispatch_run_fatal: Connection from"
# AND NOT message:"dockerd"
# AND NOT message:"avahi-autoipd"

# ES QUERY
# message: /[0-9]{2,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}/ AND NOT message:"Failed" AND NOT message:"authentication" AND NOT message:"Disconnected" AND NOT message:"disconnect" AND NOT message:"closed" AND NOT message:"receive" AND NOT message:"reset" AND NOT message:"Invalid user" AND NOT message:"publickey" AND NOT message:"password" AND NOT message:"broken pipe" AND NOT message:"Unable to negotiate" AND NOT message:"Bad protocol version" AND NOT message:"COMMAND" AND NOT message: "url_helper.py" AND NOT message:"UFW" AND NOT message:"ephemeral" AND NOT message:"Protocol major versions differ for" AND NOT message:"dhcp.py" AND NOT message:"dhclient" AND NOT message:"launch_jupyter.sh" AND NOT message:"packetbeat" AND NOT message:"filebeat" AND NOT message:"avahi-autoipd" AND NOT message:"ntpd" AND NOT message:"dhcpd" AND NOT message: "cloud-init" AND NOT message:"DHCPACK" AND NOT message: "kibana" AND NOT message: "ci-info" AND NOT message:"DHCPDISCOVER" AND NOT message:"DHCPREQUEST" AND NOT message:"DHCPOFFER" AND NOT message:"No subnet declaration" AND NOT message:"bound to" AND NOT message:"Could not write ident string to" AND NOT message:"fatal: ssh_dispatch_run_fatal: Connection from" AND NOT message:"DataSourceOpenStack.py" AND NOT message:"Resolving URL: http:"
#
#####################################################################
# Print the underlying JSON representation of your Query
#####################################################################
#res_json = res.to_dict()
#print(res_json)

#####################################################################
# Print the underlying JSON representation of your Query
#####################################################################
res_json = res.to_dict()
print(res_json)
    
#####################################################################
# Response execution
#####################################################################
response = res.execute()
#####################################################################

print("Filtered logs: %i" %res.count().value) #There is an error on the number of matched logs
print("Total number of logs: %d \n" %(response.hits.total.value))